In [ ]:
!pip install window-slider
!pip install dtaidistance
from dtaidistance import dtw
import time

In [ ]:
import pathlib
import numpy as np
import pandas as pd
import json
import math
import scipy.io
from numpy import linalg as LA
import matplotlib.pyplot as plt
import os
from os.path import join, getsize
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc, roc_auc_score, precision_recall_curve, average_precision_score, cohen_kappa_score,confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from window_slider import Slider
import sys
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
np.set_printoptions(suppress=True)
from sklearn.neighbors import lof

In [ ]:
bucket_size = 30
# bucket_size = 25
length_data=dict()


In [ ]:
def ReadDataset_withlab(_file_name, _normalize=True):

    abnormal = pd.read_csv(_file_name)
    abnormal=abnormal.values

    abnormal_data = abnormal[:,0]
    abnormal_label = abnormal[:,1]

  
    return abnormal_data,abnormal_label
    

In [ ]:
def ReadDataset_seq(_file_name, _normalize=True):


    df = pd.read_csv(_file_name,header=None)
    abnormal = df.values

    abnormal_data = abnormal[:,:-1] #change as position of class labels
#     abnormal_data = abnormal[:,1:]
    

    abnormal_label = abnormal[:,-1]
#     abnormal_label = abnormal[:,0]

    print("anomaly",(len(np.where(abnormal_label==-1)[0])))


    # Normal = 1, Abnormal = -1
    
    return abnormal_data,len(abnormal_data),abnormal_label




In [ ]:
def ReadDataset_roll(_file_name_annot,anom_size):
    bucket_size=50
    base=os.path.basename(_file_name_annot) 
    base_name=base.split('Annotations_')[1]
    file_name=os.path.join(os.path.dirname(_file_name_annot),base_name)
    label_new=[]
    value=[]
    with open(file_name) as infile:
        value = [float(line.strip('\n')) for line in infile if line]
    abnormal_data=np.array(value)
    print("org len",len(abnormal_data))
    
    with open(_file_name_annot) as infile:
            indice = [int(line.strip('\n')) for line in infile if line]
        
    abnormal_label=np.ones(len(abnormal_data))

    for i in indice:
                abnormal_label[i:i+anom_size]=-1
       
#     bucket_size=anom_size+20  
    abnorm_data=np.empty((0,bucket_size),float)
    abnorm_label=np.empty((0,bucket_size),float)
    overlap_count = 0
    slider = Slider(bucket_size,overlap_count)
    slider1 = Slider(bucket_size,overlap_count)
    slider.fit(abnormal_data) 
    slider1.fit(abnormal_label)
    plt.plot(abnormal_data)
    plt.show()
    plt.plot(abnormal_label)
    plt.show()
    while True:
        window_data1 = slider.slide() 
        window_label1= slider1.slide()
        if  (len(window_data1)<bucket_size) and len(window_data1)>bucket_size/2:
          window_data=np.pad(window_data1,[0,(bucket_size-len(window_data1))])
          window_label=np.pad(window_label1,[0,(bucket_size-len(window_label1))])
          abnorm_data= np.append(abnorm_data,[window_data], axis=0)
          abnorm_label= np.append(abnorm_label,[window_label], axis=0)
          break
        else:
          if len(window_data1)==0 or len(window_data1)<bucket_size/2:
                break
          abnorm_data=np.append(abnorm_data,[window_data1], axis=0)
          abnorm_label=np.append(abnorm_label,[window_label1], axis=0)
            
    label=np.sign(np.sum(abnorm_label,axis=1))
    combined_data=np.column_stack((label,abnorm_data))

    return combined_data, anom_count
  

In [ ]:
if __name__ == '__main__':
              anom_track=dict()
              for root, dirs, files in os.walk('/dataset/discorddataset/'):
                    for dir in dirs:                        
                        print("root=",root)
                        print("dir=",dir)  
                        anom_dict={'Marotta_Valve_Tek_14': 128,'Marotta_Valve_Tek_17': 128, 'chfdbchf15': 200, 'ann_gun_CentroidA_1': 150, 'Patient_respiration2': 150, 'Patient_respiration': 100, 'Marotta_Valve_Tek_16': 128, 'dutch_power_demand': 800}
                   
                        for file in glob.glob(str(root+dir+'/Annotations_*.txt')): 
                                        base=os.path.basename(file)
                                        base=base.split('Annotations_')[1]
                                        base=base.split('.txt')[0]
                                        print(file)
                                        abnormal_data, a_count = ReadDataset_roll(file,anom_dict[base])
                                        anom_track[base]=a_count
                                        print(np.shape(abnormal_data))
                                        np.savetxt(os.path.join(root,dir,str(base+'.csv')), abnormal_data, delimiter=',',fmt='%10.5f')
                    break

In [ ]:
if __name__ == '__main__':
    
              for root, dirs, files in os.walk('/dataset/data_warp'):
#                     dirs=['artificial_seq']  ######change label position new and old
                    dirs=['seq_data']
#                     dirs=['Wafer','HandOutlines']
#                     dirs=['DistalPhalanxOutlineCorrect','MiddlePhalanxOutlineCorrect','ProximalPhalanxOutlineCorrect','Earthquakes','PhalangesOutlinesCorrect','Strawberry','ToeSegmentation1','ToeSegmentation2','DodgerLoopGame','ECG5000','SyntheticControl','ECG200','Wafer','HandOutlines']
#                     dirs=['TwoLeadECG','Computers','Worms','Yoga','Ford','Lightning2','InsectWingbeatSound']
                    for dir in dirs:
                        
                        s_precision = []
                        s_recall = []
                        s_f1 = []
                        s_roc_auc = []
                        s_pr_auc = []
                        s_cks = []
                        
                        print("root=",root)
                        print("dir=",dir)
                        
                        for root1,dir1,files in os.walk(os.path.join(root,dir)):
                             print(files)
                             
                             for file in files: 
                                if file.endswith('.csv'):
                                    file_name = os.path.join(root,dir,file)
                                    print(file_name)
                                
                                # abnormal_data,length ,abnormal_label = ReadDataset_withlab(file_name)
                                    abnormal_data,length,abnormal_label = ReadDataset_seq(file_name)
                                    length_data[file.split('.csv')[-2]]=length
                                    print(np.shape(abnormal_data))
                                # print(np.shape(abnormal_label))
                                    file_name1=os.path.join(os.path.join(root,dir),"out_warp",str(os.path.splitext(file)[0])+"_warp.csv")
                                    np.savetxt(file_name1, abnormal_data, delimiter=',',fmt='%10.5f')
#                                     file_name2=os.path.join(os.path.join(root,dir),"out_warp",str(os.path.splitext(file)[0])+"_warp_label.csv")
#                                 # # file_name2=os.path.join(os.path.join(root,dir),"out_warp",str(os.path.splitext(file)[0])+"_warp_revlabel.csv")
#                                     np.savetxt(file_name2, abnormal_label, delimiter=',',fmt='%d')
                             break    
                    break

    

In [ ]:
def ReadDataset_rep(_file_name,rep,root,dir, _normalize=True):


    df = pd.read_csv(_file_name,header=None)
    abnormal_data = df.values

    
    return abnormal_data


renaming files

In [ ]:
if __name__ == '__main__':
              
              for root, dirs, files in os.walk('/wartem/representations/'):

                    for dir in dirs:
                        print("root=",root)
                        print("dir=",dir)
                        warp="interpolation"
                        for root1,dir1,files in os.walk(os.path.join(root,dir,warp)):
                            print(files)
                            for file in files:
                                file_name = str(dir+'_rep.csv')
                                print(file_name)
                                os.rename(root+dir+'/'+warp+'/'+file,root+dir+'/'+warp+'/'+file_name)
                                

# **NN Method warp**

In [ ]:
import time
def RunModel(_file_name,root,dir,lenth):
    rep=1
    abnormal_data = ReadDataset_rep(_file_name,rep,root,dir) 
    y_pred_arr1= np.empty((0,lenth), int)
    y_pred_1=-1*np.ones(lenth,int)
    score_1=np.zeros(lenth)
    y_pred_arr2= np.empty((0,lenth), int)

    score_2=np.zeros(lenth)
    y_pred_2=-1*np.ones(lenth,int)
    y_pred_arr3= np.empty((0,lenth), int)

    y_pred_3=-1*np.ones(lenth,int)
    score_3=np.zeros(lenth)

   
    
    d_nn1=[np.array(0) for i in range(len(abnormal_data))]
    d_nn2=[np.array(0) for i in range(len(abnormal_data))]
    d_nn3=[np.array(0) for i in range(len(abnormal_data))]
    
    # d_nn3_arg=[np.array(0) for i in range(len(abnormal_data))]
    # d_nn3_max=[np.array(0) for i in range(len(abnormal_data))]
    # d_nn3_arg_max=[np.array(0) for i in range(len(abnormal_data))]
    w1=[]
    w2=[]
    w3=[]
 
    k=0
    start_time = time.time()

    
#     clf = lof.LocalOutlierFactor(n_neighbors=1, n_jobs=2,contamination=0.1)
#     y_temp = clf._fit_predict(abnormal_data)
#     score=clf.negative_outlier_factor_
#     dist=-1*score
#     d_nn1=dist 
#     clf = lof.LocalOutlierFactor(n_neighbors=3, n_jobs=2,contamination=0.1)
#     y_temp = clf._fit_predict(abnormal_data)
#     score=clf.negative_outlier_factor_
#     dist=-1*score 
#     d_nn2=dist
#     clf = lof.LocalOutlierFactor(n_neighbors=5, n_jobs=2,contamination=0.1)
#     y_temp = clf._fit_predict(abnormal_data)
#     score=clf.negative_outlier_factor_
#     dist=-1*score
#     d_nn3=dist


    for r,t in zip(abnormal_data,range(len(abnormal_data))):                    ####knn
            neigh = NearestNeighbors(n_neighbors=5)
            neigh.fit(abnormal_data)

            d_nn1[k]=(neigh.kneighbors([r])[0])[0][1] 
            d_nn2[k]=(neigh.kneighbors([r])[0])[0][2] 
            d_nn3[k]=(neigh.kneighbors([r])[0])[0][4] 
          

            # d_nn3_max[k]=dist[-1]
            # d_nn3_arg_max[k]=dist_arg[-1]
            # # d_nn3[k]=dist[9]

            k+=1                                             ##knn
   
    return  d_nn1,d_nn2,d_nn3


if __name__ == '__main__':
              
              for root, dirs, files in os.walk('/dataset/data_warp/'):
                     dirs=['seq_data']
#                      dirs=['artificial_seq']
#                      dirs=['DistalPhalanxOutlineCorrect','MiddlePhalanxOutlineCorrect','ProximalPhalanxOutlineCorrect','Earthquakes','PhalangesOutlinesCorrect','Strawberry','ToeSegmentation1','ToeSegmentation2','DodgerLoopGame','ECG5000','SyntheticControl','ECG200','HandOutlines','Wafer']
#                     dirs=['TwoLeadECG','Computers','Worms','Yoga','Ford','Lightning2','InsectWingbeatSound']
                     for dir in dirs:
                        
                        
                        print("root=",root)
                        print("dir=",dir)
                        warp="copy"
                        dir_=os.path.join(root,dir,warp)
                        
                        for root1,dir1,files in os.walk(dir_):
                            print(files)
#                             files=['dutch_power_demand_warp_rep.csv', 'ann_gun_CentroidA_1_warp_rep.csv', 'Patient_respiration2_warp_rep.csv', 'chfdbchf15_warp_rep.csv', 'Marotta_Valve_Tek_16_warp_rep.csv', 'Patient_respiration_warp_rep.csv', 'Marotta_Valve_Tek_17_warp_rep.csv','Marotta_Valve_Tek_14_warp_rep.csv']
                            for file in files:
                              
                              
                              if file.endswith("_warp_rep.csv"): 
                                file_name = os.path.join(dir_,file)
                                print("filename",file_name)
                               
                                dist_1,dist_2,dist_3 = RunModel(file_name,root,dir,length_data[file.split('_warp_rep.csv')[-2]])
                                abnormal_data,length,abnormal_label = ReadDataset_seq(os.path.join(root,dir,str(file.split('_warp_rep.csv')[0]+'.csv')))
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_1)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_1)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                                auc_1=metrics.auc(recall,precision)
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_2)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_2)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                                auc_3=metrics.auc(recall,precision)
                                precision, recall, thresholds = metrics.precision_recall_curve(abnormal_label, dist_3)
                                fpr, tpr, thresholds = metrics.roc_curve(abnormal_label, dist_3)
                                print(metrics.auc(recall,precision),metrics.auc(fpr, tpr))
                                auc_5=metrics.auc(recall,precision)
                            
                                mean_auc=np.mean([auc_1,auc_3,auc_5])
                                print(mean_auc)

                                
                            break
                               
                     break         
                                
                                